# Experiments on vanilla NADE(Neural Autoregressive Density Estimation)
* [paper](http://proceedings.mlr.press/v15/larochelle11a/larochelle11a.pdf)
* GPU is enabled: 
> **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**


<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/drive/1tzlvd-fzBgRP0-rhSePW9Fyq6B_RSnN3?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href=""><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

# 01. Set-up

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
cd /content/gdrive/MyDrive/colab-github/STAT37792/autoregressive/nade

/content/gdrive/MyDrive/colab-github/STAT37792/autoregressive/nade


# 02. NADE architecture

In [1]:
import torch

Based on the code given written by Hugo Larochelle, updates have been made to run using torch.

In [ ]:
class AutoregressiveModel(GenerativeModel):
    """The base class for Autoregressive generative models. """

    def __init__(self, sample_fn=None):
        """Initializes a new AutoregressiveModel instance.
        Args:
            sample_fn: A fn(logits)->sample which takes sufficient statistics of a
                distribution as input and returns a sample from that distribution.
                Defaults to the Bernoulli distribution.
        """
        super().__init__()
        self._sample_fn = sample_fn or _default_sample_fn

    def _get_conditioned_on(self, n_samples, conditioned_on):
        assert (
            n_samples is not None or conditioned_on is not None
        ), 'Must provided one, and only one, of "n_samples" or "conditioned_on"'
        if conditioned_on is None:
            shape = (n_samples, self._c, self._h, self._w)
            conditioned_on = (torch.ones(shape) * -1).to(self.device)
        else:
            conditioned_on = conditioned_on.clone()
        return conditioned_on

    # TODO(eugenhotaj): This function does not handle subpixel sampling correctly.
    def sample(self, n_samples=None, conditioned_on=None):
        """Generates new samples from the model.
        Args:
            n_samples: The number of samples to generate. Should only be provided when
                `conditioned_on is None`.
            conditioned_on: A batch of partial samples to condition the generation on.
                Only dimensions with values < 0 are sampled while dimensions with
                values >= 0 are left unchanged. If 'None', an unconditional sample is
                generated.
        """
        with torch.no_grad():
            conditioned_on = self._get_conditioned_on(n_samples, conditioned_on)
            n, c, h, w = conditioned_on.shape
            for row in range(h):
                for col in range(w):
                    out = self.forward(conditioned_on)[:, :, row, col]
                    out = self._sample_fn(out).view(n, c)
                    conditioned_on[:, :, row, col] = torch.where(
                        conditioned_on[:, :, row, col] < 0,
                        out,
                        conditioned_on[:, :, row, col],
                    )
            return conditioned_on

In [ ]:
"""Implementation of Neural Autoregressive Distribution Estimator (NADE) [1].

NADE can be viewed as a one hidden layer autoencoder masked to satisfy the 
autoregressive property. This masking allows NADE to act as a generative model
by explicitly estimating p(X) as a factor of conditional probabilities, i.e,
P(X) = \prod_i^D p(X_i|X_{j<i}), where X is a feature vector and D is the 
dimensionality of X.

[1]: https://arxiv.org/abs/1605.02226
"""

import torch
from torch import distributions
from torch import nn

from pytorch_generative.models import base


class NADE(base.AutoregressiveModel):
    """The Neural Autoregressive Distribution Estimator (NADE) model."""

    def __init__(self, input_dim, hidden_dim):
        """Initializes a new NADE instance.

        Args:
            input_dim: The dimension of the input.
            hidden_dim: The dimmension of the hidden layer. NADE only supports one
                hidden layer.
        """
        super().__init__()
        self._input_dim = input_dim

        # fmt: off
        self._in_W = nn.Parameter(torch.zeros(hidden_dim, self._input_dim))
        self._in_b = nn.Parameter(torch.zeros(hidden_dim,))
        self._h_W = nn.Parameter(torch.zeros(self._input_dim, hidden_dim))
        self._h_b = nn.Parameter(torch.zeros(self._input_dim,))
        # fmt: on
        nn.init.kaiming_normal_(self._in_W)
        nn.init.kaiming_normal_(self._h_W)

    def _forward(self, x):
        """Computes the forward pass and samples a new output.

        Returns:
            (p_hat, x_hat) where p_hat is the probability distribution over dimensions
            and x_hat is sampled from p_hat.
        """
        # If the input is an image, flatten it during the forward pass.
        original_shape = x.shape
        if len(x.shape) > 2:
            x = x.view(original_shape[0], -1)

        p_hat, x_hat = [], []
        batch_size = 1 if x is None else x.shape[0]
        # Only the bias is used to compute the first hidden unit so we must replicate it
        # to account for the batch size.
        a = self._in_b.expand(batch_size, -1)
        for i in range(self._input_dim):
            h = torch.relu(a)
            p_i = torch.sigmoid(h @ self._h_W[i : i + 1, :].t() + self._h_b[i : i + 1])
            p_hat.append(p_i)

            # Sample 'x' at dimension 'i' if it is not given.
            x_i = x[:, i : i + 1]
            x_i = torch.where(x_i < 0, distributions.Bernoulli(probs=p_i).sample(), x_i)
            x_hat.append(x_i)

            # We do not need to add self._in_b[i:i+1] when computing the other hidden
            # units since it was already added when computing the first hidden unit.
            a = a + x_i @ self._in_W[:, i : i + 1].t()
        if x_hat:
            return (
                torch.cat(p_hat, dim=1).view(original_shape),
                torch.cat(x_hat, dim=1).view(original_shape),
            )
        return []

    def forward(self, x):
        """Computes the forward pass.

        Args:
            x: Either a tensor of vectors with shape (n, input_dim) or images with shape
                (n, 1, h, w) where h * w = input_dim.
        Returns:
            The result of the forward pass.
        """
        return self._forward(x)[0]

    def sample(self, n_samples=None, conditioned_on=None):
        """See the base class."""
        with torch.no_grad():
            conditioned_on = self._get_conditioned_on(n_samples, conditioned_on)
            return self._forward(conditioned_on)[1]


def reproduce(
    n_epochs=50,
    batch_size=512,
    log_dir="/tmp/run",
    n_gpus=1,
    device_id=0,
    debug_loader=None,
):
    """Training script with defaults to reproduce results.

    The code inside this function is self contained and can be used as a top level
    training script, e.g. by copy/pasting it into a Jupyter notebook.

    Args:
        n_epochs: Number of epochs to train for.
        batch_size: Batch size to use for training and evaluation.
        log_dir: Directory where to log trainer state and TensorBoard summaries.
        n_gpus: Number of GPUs to use for training the model. If 0, uses CPU.
        device_id: The device_id of the current GPU when training on multiple GPUs.
        debug_loader: Debug DataLoader which replaces the default training and
            evaluation loaders if not 'None'. Do not use unless you're writing unit
            tests.
    """
    from torch import optim
    from torch.nn import functional as F

    from pytorch_generative import datasets
    from pytorch_generative import models
    from pytorch_generative import trainer

    train_loader, test_loader = debug_loader, debug_loader
    if train_loader is None:
        train_loader, test_loader = datasets.get_mnist_loaders(
            batch_size, dynamically_binarize=True
        )

    model = models.NADE(input_dim=784, hidden_dim=500)
    optimizer = optim.Adam(model.parameters())

    def loss_fn(x, _, preds):
        batch_size = x.shape[0]
        x, preds = x.view((batch_size, -1)), preds.view((batch_size, -1))
        loss = F.binary_cross_entropy_with_logits(preds, x, reduction="none")
        return loss.sum(dim=1).mean()

    model_trainer = trainer.Trainer(
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        eval_loader=test_loader,
        log_dir=log_dir,
        n_gpus=n_gpus,
        device_id=device_id,
    )
    model_trainer.interleaved_train_and_eval(n_epochs)

In [ ]:
import torch
import mlpython.learners.generic as mlgeneric
import mlpython.mathutils.nonlinear as mlnonlin
import mlpython.mathutils.linalg as mllin
import numpy as np

class NADE(mlgeneric.OnlineLearner):
   """
   Neural Autoregressive Distribution Estimator (NADE) for multivariate binary distribution estimation
   """

   def initialize_learner(self,metadata):
      self.rng = np.random.mtrand.RandomState(self.seed)
      self.input_size = metadata['input_size']
      if self.hidden_size <= 0:
          raise ValueError('hidden_size should be > 0')

      self.W = (2*self.rng.rand(self.hidden_size,self.input_size)-1)/self.input_size
      self.c = np.zeros((self.hidden_size))
      self.b = np.zeros((self.input_size))

      self.dW = np.zeros((self.hidden_size,self.input_size))
      self.dc = np.zeros((self.hidden_size))
      self.db = np.zeros((self.input_size))

      if self.untied_weights:
          self.V = (2*self.rng.rand(self.hidden_size,self.input_size)-1)/self.input_size
          self.dV = np.zeros((self.hidden_size,self.input_size))

      self.input = np.zeros((self.input_size))
      self.input_times_W = np.zeros((self.hidden_size,self.input_size))
      self.acc_input_times_W = np.zeros((self.hidden_size,self.input_size))
      self.hid = np.zeros((self.hidden_size,self.input_size))
      self.Whid = np.zeros((self.hidden_size,self.input_size))
      self.recact = np.zeros((self.input_size))
      self.rec = np.zeros((self.input_size))
      
      self.dinput_times_W = np.zeros((self.hidden_size,self.input_size))
      self.dacc_input_times_W = np.zeros((self.hidden_size,self.input_size))
      self.dhid = np.zeros((self.hidden_size,self.input_size))
      self.dWhid = np.zeros((self.hidden_size,self.input_size))
      self.dWenc = np.zeros((self.hidden_size,self.input_size))
      self.drecact = np.zeros((self.input_size))
      self.drec = np.zeros((self.input_size))
      
      self.n_updates = 0

   def update_learner(self,example):
      self.input[self.input_order] = example
   
      # fprop
      np.multiply(self.input,self.W,self.input_times_W)
      np.add.accumulate(self.input_times_W[:,:-1],axis=1,out=self.acc_input_times_W[:,1:])
      self.acc_input_times_W[:,0] = 0
      self.acc_input_times_W += self.c[:,np.newaxis]
      mlnonlin.sigmoid(self.acc_input_times_W,self.hid)

      if self.untied_weights:
          np.multiply(self.hid,self.V,self.Whid)
      else:
          np.multiply(self.hid,self.W,self.Whid)

      mllin.sum_columns(self.Whid,self.recact)
      self.recact += self.b
      mlnonlin.sigmoid(self.recact,self.rec)

      # bprop
      np.subtract(self.rec,self.input,self.drec)
      self.db[:] = self.drec

      if self.untied_weights:
          np.multiply(self.drec,self.hid,self.dV)
          np.multiply(self.drec,self.V,self.dhid)
          self.dW[:] = 0
      else:
          np.multiply(self.drec,self.hid,self.dW)
          np.multiply(self.drec,self.W,self.dhid)

      mlnonlin.dsigmoid(self.hid,self.dhid,self.dacc_input_times_W)
      mllin.sum_rows(self.dacc_input_times_W,self.dc)      
      np.add.accumulate(self.dacc_input_times_W[:,:0:-1],axis=1,out=self.dWenc[:,-2::-1])
      self.dWenc[:,-1] = 0
      self.dWenc *= self.input
      self.dW += self.dWenc

      self.dW *= self.learning_rate/(1.+self.decrease_constant*self.n_updates)
      self.db *= self.learning_rate/(1.+self.decrease_constant*self.n_updates)
      self.dc *= self.learning_rate/(1.+self.decrease_constant*self.n_updates)

      self.W -= self.dW
      self.b -= self.db
      self.c -= self.dc

      if self.untied_weights:
          self.dV *= self.learning_rate/(1.+self.decrease_constant*self.n_updates)
          self.V -= self.dV
      self.n_updates += 1

   def use_learner(self,example):
      self.input[self.input_order] = example
      output = np.zeros((self.input_size))
      recact = np.zeros((self.input_size))
   
      # fprop
      np.multiply(self.input,self.W,self.input_times_W)
      np.add.accumulate(self.input_times_W[:,:-1],axis=1,out=self.acc_input_times_W[:,1:])
      self.acc_input_times_W[:,0] = 0
      self.acc_input_times_W += self.c[:,np.newaxis]
      mlnonlin.sigmoid(self.acc_input_times_W,self.hid)
      if self.untied_weights:
          np.multiply(self.hid,self.V,self.Whid)
      else:
          np.multiply(self.hid,self.W,self.Whid)

      mllin.sum_columns(self.Whid,recact)
      recact += self.b
      mlnonlin.sigmoid(recact,output)
      return [output,recact]

   def cost(self,outputs,example):
      self.input[self.input_order] = example
      #return [ np.sum(-self.input*np.log(outputs[0]) - (1-self.input)*np.log(1-outputs[0])) ]
      return [ np.sum(-self.input*(outputs[1]-np.log(1+np.exp(outputs[1]))) - (1-self.input)*(-outputs[1]-np.log(1+np.exp(-outputs[1])))) ]

   def sample(self):
      input = np.zeros(self.input_size)
      input_prob = np.zeros(self.input_size)
      hid_i = np.zeros(self.hidden_size)
      for i in range(self.input_size):
         if i > 0:
            mlnonlin.sigmoid(self.c+np.dot(self.W[:,:i],input[:i]),hid_i)
         else:
            mlnonlin.sigmoid(self.c,hid_i)

         if self.untied_weights:
            mlnonlin.sigmoid(np.dot(hid_i,self.V[:,i])+self.b[i:i+1],input_prob[i:i+1])
         else:
            mlnonlin.sigmoid(np.dot(hid_i,self.W[:,i])+self.b[i:i+1],input_prob[i:i+1])

         input[i] = (self.rng.rand()<input_prob[i])

      return (input[self.input_order],input_prob[self.input_order])

   def verify_gradients(self,untied_weights):
      
      print('WARNING: calling verify_gradients reinitializes the learner')

      rng = np.random.mtrand.RandomState(1234)
      input_order = range(20)
      rng.shuffle(input_order)

      self.seed = 1234
      self.hidden_size = 10
      self.input_order = input_order
      self.untied_weights = untied_weights
      self.initialize_learner({'input_size':20})
      example = rng.rand(20)<0.5
      epsilon=1e-6
      self.learning_rate = 1
      self.decrease_constant = 0

      W_copy = np.array(self.W)
      emp_dW = np.zeros(self.W.shape)
      for i in range(self.W.shape[0]):
         for j in range(self.W.shape[1]):
            self.W[i,j] += epsilon
            output = self.use_learner(example)
            a = self.cost(output,example)[0]
            self.W[i,j] -= epsilon

            self.W[i,j] -= epsilon
            output = self.use_learner(example)
            b = self.cost(output,example)[0]
            self.W[i,j] += epsilon

            emp_dW[i,j] = (a-b)/(2.*epsilon)

      self.update_learner(example)
      self.W[:] = W_copy
      print('dW diff.:',np.sum(np.abs(self.dW.ravel()-emp_dW.ravel()))/self.W.ravel().shape[0])

      b_copy = np.array(self.b)
      emp_db = np.zeros(self.b.shape)
      for i in range(self.b.shape[0]):
         self.b[i] += epsilon
         output = self.use_learner(example)
         a = self.cost(output,example)[0]
         self.b[i] -= epsilon
         
         self.b[i] -= epsilon
         output = self.use_learner(example)
         b = self.cost(output,example)[0]
         self.b[i] += epsilon
         
         emp_db[i] = (a-b)/(2.*epsilon)

      self.update_learner(example)
      self.b[:] = b_copy
      print('db diff.:',np.sum(np.abs(self.db.ravel()-emp_db.ravel()))/self.b.ravel().shape[0])

      c_copy = np.array(self.c)
      emp_dc = np.zeros(self.c.shape)
      for i in range(self.c.shape[0]):
         self.c[i] += epsilon
         output = self.use_learner(example)
         a = self.cost(output,example)[0]
         self.c[i] -= epsilon

         self.c[i] -= epsilon
         output = self.use_learner(example)
         b = self.cost(output,example)[0]
         self.c[i] += epsilon

         emp_dc[i] = (a-b)/(2.*epsilon)

      self.update_learner(example)
      self.c[:] = c_copy
      print('dc diff.:',np.sum(np.abs(self.dc.ravel()-emp_dc.ravel()))/self.c.ravel().shape[0])

      if untied_weights:
         V_copy = np.array(self.V)
         emp_dV = np.zeros(self.V.shape)
         for i in range(self.V.shape[0]):
            for j in range(self.V.shape[1]):
               self.V[i,j] += epsilon
               output = self.use_learner(example)
               a = self.cost(output,example)[0]
               self.V[i,j] -= epsilon
         
               self.V[i,j] -= epsilon
               output = self.use_learner(example)
               b = self.cost(output,example)[0]
               self.V[i,j] += epsilon
         
               emp_dV[i,j] = (a-b)/(2.*epsilon)
         
         self.update_learner(example)
         self.V[:] = V_copy
         print('dV diff.:',np.sum(np.abs(self.dV.ravel()-emp_dV.ravel()))/self.V.ravel().shape[0])




# 03. Training

# 04. Extension: What if using ReLU instead of Sigmoid?